In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
#source: https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_flan_t5_large_bnb_peft.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q bitsandbytes datasets accelerate
!pip install transformers
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Select CUDA device index
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-large"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# **Prepare model for training**
Some pre-processing needs to be done before training such an int8 model using peft, therefore let's import an utiliy function prepare_model_for_int8_training that will:

Casts all the non int8 modules to full precision (fp32) for stability
Add a forward_hook to the input embedding layer to enable gradient computation of the input hidden states
Enable gradient checkpointing for more memory-efficient training

In [ ]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Load your PeftModel
Here we will use LoRA (Low-Rank Adaptators) to train our model

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM"
)


model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 787868672 || trainable%: 0.5989059049678777


# Load and process data

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

import torch
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils import resample

import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.model_selection import train_test_split

from datasets import Dataset

Label df

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

peft_model_id = "kenchenxingyu/flan-large-lora-stance-human7"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [ ]:
origin_data = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/origin_data_NER_emotion_stance.h5',compression='xz')
origin_data['source_text'] = "Classify stance towards the target:" + origin_data['target'].apply(lambda x: str(x)) + " in the text: "  + origin_data['post_text']
origin_data

,create_date,tweet_id,similarity,like_count,retweet_count,post_text,claim_number,group,hashtag,tweet_user,NER,target_type,target,target_filter,source_text,flan_emotion_pred,emotion,flan_stance,stance
0,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952,"(the Miami Marlins, ORG)",ORG,the Miami Marlins,True,Classify stance towards the target:the Miami M...,4,neutral,0,AGAINST
1,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952,"(ESPN, ORG)",ORG,ESPN,True,Classify stance towards the target:ESPN in the...,4,neutral,1,NEUTRAL
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(Hialeah, GPE)",GPE,Hialeah,True,Classify stance towards the target:Hialeah in ...,4,neutral,1,NEUTRAL
3,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(FL, GPE)",GPE,FL,True,Classify stance towards the target:FL in the t...,4,neutral,1,NEUTRAL
4,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"(Carlos Hernandez, PERSON)",PERSON,Carlos Hernandez,True,Classify stance towards the target:Carlos Hern...,4,neutral,1,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,2020-12-05T11:19:16.000Z,1335181945635004417,0.659882,1436,180,We broke the news on #OutLoudWithGiannoCaldwel...,298,1,OutLoudWithGiannoCaldwell,73973,"(Gianno Caldwell.https://t.co/aAKVBelRJO, PERSON)",PERSON,Gianno Caldwell.https://t.co/aAKVBelRJO,True,Classify stance towards the target:Gianno Cald...,4,neutral,2,FAVOR
3778,2020-12-20T20:31:55.000Z,1340756841454850049,0.390273,171932,2654,I see your tweets and I’m trying to double and...,298,1,NaN,75482,"(MrBeast Burgers, ORG)",ORG,MrBeast Burgers,True,Classify stance towards the target:MrBeast Bur...,4,neutral,2,FAVOR
3779,2020-05-08T17:16:17.000Z,1258807951164239873,0.856750,283,205,A 2020 peer-reviewed study published in the jo...,299,1,NaN,103443,"(US, GPE)",GPE,US,True,Classify stance towards the target:US in the t...,4,neutral,1,NEUTRAL
3780,2020-05-08T17:16:17.000Z,1258807951164239873,0.856750,283,205,A 2020 peer-reviewed study published in the jo...,299,1,NaN,103443,"(DoD, ORG)",ORG,DoD,True,Classify stance towards the target:DoD in the ...,4,neutral,1,NEUTRAL


In [ ]:
from IPython.display import clear_output

df = origin_data

samples_number = len(df)
progress_bar = tqdm(range(samples_number))
predictions_list = []


for i in range(samples_number):
    input_text = df['source_text'][i]

    if len(input_text.split()) < 1000:
        inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True,return_tensors="pt")
    else:
        inputs = tokenizer(input_text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

    # Move the input tensors to the GPU if they are not already there
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}
    #print(inputs)

    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             attention_mask=inputs['attention_mask'].to('cuda'),
                             max_length=10,
                             #num_beams=4,
                             #early_stopping=True
                             )
    del(inputs)
    torch.cuda.empty_cache()
    #gc.collect()

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_list.append(prediction)
    #labels_list.append(test_df['label'][i])
    clear_output(wait=True)
    progress_bar.update(1)

origin_data['flan_stance'] = predictions_list

reverse_stance_mapping = {0:'AGAINST',
                  1:'NEUTRAL',
                  2:'FAVOR',
}

origin_data['stance'] = origin_data['flan_stance'].apply(lambda x: int(x)).map(reverse_stance_mapping)

origin_data.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/origin_data_NER_emotion_stance.h5',compression='xz')


100%|██████████| 3782/3782 [10:53<00:00,  6.39it/s]

In [ ]:
origin_data.stance.value_counts(dropna=False)

NEUTRAL    2083
AGAINST    1455
FAVOR       244
Name: stance, dtype: int64

In [ ]:
news_claim = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim_NER_emotion_stance.h5',compression='xz')
news_claim['source_text'] = "Classify stance towards the target:" + news_claim['target'].apply(lambda x: str(x)) + " in the text: "  + news_claim['text']
news_claim

,text,source,claim_num,group,NER,target_type,target,target_filter,source_text,flan_emotion_pred,emotion
0,every election year has a disease sars in 200...,politifact,1,1,"(zika, PERSON)",PERSON,zika,True,Classify stance towards the target:zika in the...,4,neutral
1,obama ordered cia to train isis jihadists,politifact,2,3,"(obama, PERSON)",PERSON,obama,True,Classify stance towards the target:obama in th...,4,neutral
2,obama ordered cia to train isis jihadists,politifact,2,3,"(cia, ORG)",ORG,cia,True,Classify stance towards the target:cia in the ...,4,neutral
3,"says george soros said, im going to bring down...",politifact,3,3,"(george soros, PERSON)",PERSON,george soros,True,Classify stance towards the target:george soro...,4,neutral
4,"says george soros said, im going to bring down...",politifact,3,3,"(the united states, GPE)",GPE,the united states,True,Classify stance towards the target:the united ...,4,neutral
...,...,...,...,...,...,...,...,...,...,...,...
1604,nancy pelosi may have just committed a violati...,politifact,1349,3,"(trumps, PERSON)",PERSON,trumps,True,Classify stance towards the target:trumps in t...,4,neutral
1605,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"(donald trump, PERSON)",PERSON,donald trump,True,Classify stance towards the target:donald trum...,4,neutral
1606,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"(dnc, ORG)",ORG,dnc,True,Classify stance towards the target:dnc in the ...,4,neutral
1607,says eight iowa counties have more adults regi...,politifact,1351,3,"(iowa, GPE)",GPE,iowa,True,Classify stance towards the target:iowa in the...,4,neutral


In [ ]:
from IPython.display import clear_output

df = news_claim

samples_number = len(df)
progress_bar = tqdm(range(samples_number))
predictions_list = []


for i in range(samples_number):
    input_text = df['source_text'][i]

    if len(input_text.split()) < 1000:
        inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True,return_tensors="pt")
    else:
        inputs = tokenizer(input_text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

    # Move the input tensors to the GPU if they are not already there
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}
    #print(inputs)

    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             attention_mask=inputs['attention_mask'].to('cuda'),
                             max_length=10,
                             #num_beams=4,
                             #early_stopping=True
                             )
    del(inputs)
    torch.cuda.empty_cache()
    #gc.collect()

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_list.append(prediction)
    #labels_list.append(test_df['label'][i])
    clear_output(wait=True)
    progress_bar.update(1)

news_claim['flan_stance'] = predictions_list

reverse_stance_mapping = {0:'AGAINST',
                  1:'NEUTRAL',
                  2:'FAVOR',
}

news_claim['stance'] = news_claim['flan_stance'].apply(lambda x: int(x)).map(reverse_stance_mapping)

news_claim.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim_NER_emotion_stance.h5',compression='xz')

100%|██████████| 1609/1609 [04:27<00:00,  5.81it/s]

In [ ]:
news_claim.stance.value_counts(dropna=False)

NEUTRAL    941
AGAINST    572
FAVOR       96
Name: stance, dtype: int64

In [ ]:
claim_propagation = pd.read_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation_data_NER_emotion_stance.h5',compression='xz')
claim_propagation['source_text'] = "Classify stance towards the target:" + claim_propagation['target'].apply(lambda x: str(x)) + " in the text: "  + claim_propagation['post_text']
claim_propagation

,tweet_user,tweet_id,like_count,depth,parent_user,create_date,parent_id,retweet_count,post_text,claim_number,group,hashtag,NER,target_type,target,target_filter,source_text,flan_emotion_pred,emotion
0,64,1260590991641362436,301,0,0,2020-05-13T15:21:27.000Z,0,113,So you left a 69 page guidebook but no PPE or ...,141,1,NaN,"(PPE, ORG)",ORG,PPE,True,Classify stance towards the target:PPE in the ...,4,neutral
1,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN,"(NHS, ORG)",ORG,NHS,True,Classify stance towards the target:NHS in the ...,4,neutral
2,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN,"(wards &amp, ORG)",ORG,wards &amp,True,Classify stance towards the target:wards &amp ...,4,neutral
3,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN,"(@GovTimWalz, PERSON)",PERSON,@GovTimWalz,True,Classify stance towards the target:@GovTimWalz...,4,neutral
4,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN,"(Minnesotans, ORG)",ORG,Minnesotans,True,Classify stance towards the target:Minnesotans...,4,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289116,144736,1235937681047080960,0,1,106463,2020-03-06T14:38:00.000Z,1235932698688053249,0,"#WaterIsLife \nremember your vote matters, \nC...",229,2,WaterIsLife#WhenWeAllVote#FlintWaterCrisis,NaN,,[],True,Classify stance towards the target:[] in the t...,4,neutral
289117,144738,1242264347025571842,0,1,93248,2020-03-24T01:37:55.000Z,1242257594540056576,1,NO @GOP should even entertain this anti-Americ...,96,0,NaN,"(@realDonaldTrump @senatemajldr, ORG)",ORG,@realDonaldTrump @senatemajldr,True,Classify stance towards the target:@realDonald...,4,neutral
289118,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN,"(AMERICAN IMPERIALISM, ORG)",ORG,AMERICAN IMPERIALISM,True,Classify stance towards the target:AMERICAN IM...,4,neutral
289119,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN,"(https://t.co/JFI65K1n1U, ORG)",ORG,https://t.co/JFI65K1n1U,True,Classify stance towards the target:https://t.c...,4,neutral


In [ ]:
from IPython.display import clear_output

df = claim_propagation

samples_number = len(df)
progress_bar = tqdm(range(samples_number))
predictions_list = []


for i in range(samples_number):
    input_text = df['source_text'][i]

    if len(input_text.split()) < 1000:
        inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True,return_tensors="pt")
    else:
        inputs = tokenizer(input_text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

    # Move the input tensors to the GPU if they are not already there
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}
    #print(inputs)

    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             attention_mask=inputs['attention_mask'].to('cuda'),
                             max_length=10,
                             #num_beams=4,
                             #early_stopping=True
                             )
    del(inputs)
    torch.cuda.empty_cache()
    #gc.collect()

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_list.append(prediction)
    #labels_list.append(test_df['label'][i])
    clear_output(wait=True)
    progress_bar.update(1)

claim_propagation['flan_stance'] = predictions_list

reverse_stance_mapping = {0:'AGAINST',
                  1:'NEUTRAL',
                  2:'FAVOR',
}

claim_propagation['stance'] = claim_propagation['flan_stance'].apply(lambda x: int(x)).map(reverse_stance_mapping)

100%|██████████| 289121/289121 [13:15:55<00:00,  6.59it/s]

In [ ]:
claim_propagation.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation_data_NER_emotion_stance.h5',compression='xz')

In [ ]:
from google.colab import runtime
runtime.unassign()